In [1]:
import pandas as pd

In [2]:
## 구글 드리아브 mount
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## 데이터 load
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/review_modify.csv')
data.head()

,식당이름,리뷰,리뷰_리스트
0,(휴업중) 유라꾸키친,달달한 김치나베돈까츠가 맛있는 곳 입니다 최근에 무인결제 기계가 생겨서 결제도 쉽고...,['달달한 김치나베돈까츠가 맛있는 곳 입니다 최근에 무인결제 기계가 생겨서 결제도 ...
1,(휴업중)병우네,삼청동 골목에 숨어있는 민어집입니다 궁금했었는데 재방문 기회가 생겼네요 골목 안...,['삼청동 골목에 숨어있는 민어집입니다 궁금했었는데 재방문 기회가 생겼네요 골목...
2,010수산,가성비 좋은 무난한 횟집 우럭매운탕 사람들이 많이 시킴 회 한점 간단히 먹고 싶을 ...,['가성비 좋은 무난한 횟집 우럭매운탕 사람들이 많이 시킴 회 한점 간단히 먹고 싶...
3,03수제비칼국수,집밥을 향해 돌진하는 방배 마을 회관 같은 식당 언제든 들려 한 끼 푸짐한 밥을 ...,['집밥을 향해 돌진하는 방배 마을 회관 같은 식당 언제든 들려 한 끼 푸짐한 밥...
4,05식당,마늘마늘해서 징글징글하다 맵두 주문 두부랑 고기랑 김치랑 들어가서 밥 한공...,['마늘마늘해서 징글징글하다 맵두 주문 두부랑 고기랑 김치랑 들어가서 밥 ...


In [4]:
## 리뷰리스트 데이터 str -> list
import ast
data['리뷰_리스트']= data['리뷰_리스트'].apply(ast.literal_eval)

In [5]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [6]:
ls

drive/  Mecab-ko-for-Google-Colab/  sample_data/


In [7]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


# Tokenizer

In [8]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 460kB 57.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-03-19 09:21:03--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22c2:513, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=cRgTRZic%2FsO0eVXe7jGScqhm9I4%3D&Expires=1616147463&AWSAccessKeyId=AKIA6KOSE3B

In [185]:
from konlpy.tag import Mecab
mecab = Mecab()

def tokenizer(row):
  return mecab.morphs(row)

In [186]:
data['리뷰_토큰'] = data['리뷰'].apply(tokenizer)
data.head()

,식당이름,리뷰,리뷰_리스트,리뷰_토큰
0,(휴업중) 유라꾸키친,달달한 김치나베돈까츠가 맛있는 곳 입니다 최근에 무인결제 기계가 생겨서 결제도 쉽고...,[달달한 김치나베돈까츠가 맛있는 곳 입니다 최근에 무인결제 기계가 생겨서 결제도 쉽...,"[달달, 한, 김치, 나베, 돈, 까, 츠, 가, 맛있, 는, 곳, 입니다, 최근,..."
1,(휴업중)병우네,삼청동 골목에 숨어있는 민어집입니다 궁금했었는데 재방문 기회가 생겼네요 골목 안...,[삼청동 골목에 숨어있는 민어집입니다 궁금했었는데 재방문 기회가 생겼네요 골목 ...,"[삼청동, 골목, 에, 숨, 어, 있, 는, 민어, 집, 입니다, 궁금, 했었, 는..."
2,010수산,가성비 좋은 무난한 횟집 우럭매운탕 사람들이 많이 시킴 회 한점 간단히 먹고 싶을 ...,[가성비 좋은 무난한 횟집 우럭매운탕 사람들이 많이 시킴 회 한점 간단히 먹고 싶을...,"[가성, 비, 좋, 은, 무난, 한, 횟집, 우럭, 매운탕, 사람, 들, 이, 많이..."
3,03수제비칼국수,집밥을 향해 돌진하는 방배 마을 회관 같은 식당 언제든 들려 한 끼 푸짐한 밥을 ...,[집밥을 향해 돌진하는 방배 마을 회관 같은 식당 언제든 들려 한 끼 푸짐한 밥을...,"[집, 밥, 을, 향해, 돌진, 하, 는, 방배, 마을, 회관, 같, 은, 식당, ..."
4,05식당,마늘마늘해서 징글징글하다 맵두 주문 두부랑 고기랑 김치랑 들어가서 밥 한공...,[마늘마늘해서 징글징글하다 맵두 주문 두부랑 고기랑 김치랑 들어가서 밥 한...,"[마늘, 마늘, 해서, 징글징글, 하, 다, 맵, 두, 주문, 두부, 랑, 고기, ..."


In [12]:
sentences = data['리뷰_토큰']

Gensim Word2Vec

In [187]:
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
 
# 문장을 이용하여 단어와 벡터를 생성한다.
model = Word2Vec(sentences, size=300, window=5, min_count=5, workers=3)
 
# 단어벡터를 구한다.
word_vectors = model.wv
 
vocabs            = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs]
 



In [189]:
len(vocabs)
# word_vectors_list[0]
len(word_vectors_list[0])

300

In [212]:
# 결과1 : 두개의 단어간 유사도를 확인
print(word_vectors.similarity(w1='바삭바삭', w2='촉촉'))
 

0.7662661


In [213]:
model.wv.most_similar(positive=['바삭바삭'],  negative=[], topn=50, restrict_vocab=None)

[('바삭', 0.8987230062484741),
 ('쫀득쫀득', 0.8111810684204102),
 ('쫄깃쫄깃', 0.795353889465332),
 ('빠삭', 0.790899395942688),
 ('쫀득', 0.768020749092102),
 ('촉촉', 0.7662662267684937),
 ('부들부들', 0.7577285766601562),
 ('보들보들', 0.7481569051742554),
 ('쫄깃', 0.7366982698440552),
 ('폭신폭신', 0.7314518690109253),
 ('포슬포슬', 0.7196090221405029),
 ('말랑말랑', 0.7100765705108643),
 ('쫀쫀', 0.7051758766174316),
 ('탱글탱글', 0.7032670974731445),
 ('따끈따끈', 0.700066089630127),
 ('야들야들', 0.6982064247131348),
 ('도톰', 0.6923424601554871),
 ('파삭', 0.6916287541389465),
 ('두툼', 0.6874081492424011),
 ('폭신', 0.6797157526016235),
 ('딱딱', 0.6721928119659424),
 ('따끈', 0.6610658168792725),
 ('눅눅', 0.659677267074585),
 ('통통', 0.6532432436943054),
 ('고소', 0.6492085456848145),
 ('짭조름', 0.6456612348556519),
 ('탱탱', 0.644561767578125),
 ('달짝지근', 0.6429345011711121),
 ('단단', 0.6413952112197876),
 ('담백', 0.6367602348327637),
 ('꾸덕꾸덕', 0.6352525353431702),
 ('말랑', 0.6285505890846252),
 ('뻑뻑', 0.6251176595687866),
 ('찐득', 0.62426996231079

## 전이학습

In [ ]:
## 사전학습된 데이터 형식이 안맞는지 error발생ㅠ 
file_name = '/content/drive/MyDrive/Colab_Notebooks/ko.tsv'
model.intersect_word2vec_format(fname=file_name, binary=False,  unicode_errors='strict')

In [171]:
# 단어벡터를 구한다.
word_vectors = model_300.wv
 
vocabs            = word_vectors.vocab.keys()
word_vectors_list = [word_vectors[v] for v in vocabs]

In [82]:
# 최종 모델을 저장한다.
model.save('review_500.model')

In [72]:
# 저장한 모델을 읽어서 이용한다.
model_300 = Word2Vec.load('review_300.model')

In [83]:
# 저장한 모델을 읽어서 이용한다.
model_500 = Word2Vec.load('review_500.model')

In [68]:
# 저장한 모델을 읽어서 이용한다.
model_1000 = Word2Vec.load('review_1000.model')

In [ ]:
model_300.wv.most_similar(positive=['바삭바삭'], negative=[], topn=50, restrict_vocab=None)